In [2]:
import pandas as pd
import numpy as np

In [20]:

# Read data ...
costs_df_raw = pd.read_csv('../../repository/daily_usage_by_service.csv')


## Data table

In [5]:
def costs_per_month(column):
  """
  Given a dataframe column create a pivoted table with the costs per month split by 
  the column input.
  """
  # Parameters
  index_cols = ['som']
  value_cols = ['amount_usd']
  index_cols.append(column)

  # Add columns
  costs_df = (
      costs_df_raw
      .pipe(lambda x: x.assign(date=pd.to_datetime(x['date'], format='%d/%m/%Y')))
  )

  costs_df['som'] = costs_df['date'].dt.to_period('M').dt.to_timestamp()
  costs_df['date'] = costs_df['date'].dt.date

  # This pivot funciton errors for some reason
  pivot_long = pd.pivot_table(
      data=costs_df,
      index=index_cols,
      values=value_cols,
      aggfunc=np.sum
  ).reset_index()

  pivot_wide = pivot_long.pivot(index='som', columns=[column], values=[
                                'amount_usd']).reset_index()

  # Rename columns, streamlit doesn't currently support multi-level indexes
  new_names = []
  for i, colname in enumerate(pivot_wide.columns):
    if colname[0] == '':
      new_names.append(colname[1])
    elif colname[1] == '':
      new_names.append(colname[0])
    else:
      # new_names.append('_'.join(colname).strip())
      new_names.append(colname[1])

  pivot_wide.columns = new_names

  # Format df
  pivot_wide = (
      pivot_wide
      .pipe(lambda x: x.fillna(0))
      .pipe(lambda x: x.sort_values(by='som', ascending=False))
      .pipe(lambda x: x.assign(som=x.som.dt.strftime('%b-%Y')))
  ).set_index('som')

  pivot_wide = pivot_wide.style.format('${0:,.2f}')

  return pivot_wide

In [ ]:
costs_per_month('account_type')

## Line chart

,AWS,Azure,GCP
som,,,
Jan-2022,253.18,33.96,0.57
Dec-2021,375.43,46.33,0.32
Nov-2021,360.47,108.48,0.00
Oct-2021,458.19,0.00,0.00
Sep-2021,362.24,0.00,0.00
Aug-2021,264.42,0.00,0.00
Jul-2021,71.52,0.00,0.00
